# TA Scheduling

## Imports

In [14]:
!pip install ortools

In [15]:
from dataclasses import dataclass, field
from typing import Annotated, List, Dict, Optional
from datetime import datetime, time, timedelta
import pandas as pd
import numpy as np
import os
from ortools.sat.python import cp_model
print("Importing was successful")


Importing was successful


## Naive Programming

In [ ]:
def schedule_tas_debug(availability, ta_requirements, shift_requirements):
    num_tas = len(availability)
    num_shifts = len(availability[0])

    print("Availability Matrix:")
    for i in range(num_tas):
        print(f"TA {i}: {availability[i]}")

    print("\nTA Requirements:", ta_requirements)
    print("Shift Requirements:", shift_requirements)

    # Create the CP-SAT model
    model = cp_model.CpModel()

    # Decision variables: x[i][j] is 1 if TA i is assigned to shift j, else 0
    x = {}
    for i in range(num_tas):
        for j in range(num_shifts):
            x[i, j] = model.NewBoolVar(f'x[{i}][{j}]')

    # Constraint: TAs must meet their shift requirements
    for i in range(num_tas):
        model.Add(sum(x[i, j] for j in range(num_shifts)) == ta_requirements[i])
        print(f"Added TA {i} shift requirement: {ta_requirements[i]}")

    # Constraint: Shifts must meet staffing requirements
    for j in range(num_shifts):
        model.Add(sum(x[i, j] for i in range(num_tas)) == shift_requirements[j])
        print(f"Added shift {j} staffing requirement: {shift_requirements[j]}")

    # Constraint: Respect TA availability
    for i in range(num_tas):
        for j in range(num_shifts):
            if availability[i][j] == -1:  # Unavailable
                model.Add(x[i, j] == 0)
                print(f"TA {i} cannot work shift {j}")

    # Objective: Maximize TA satisfaction
    objective_terms = []
    for i in range(num_tas):
        for j in range(num_shifts):
            if availability[i][j] == 1:  # Desired
                objective_terms.append(10 * x[i, j])  # High weight for desired
            elif availability[i][j] == 0:  # Undesired
                objective_terms.append(-1 * x[i, j])  # Low weight for undesired
    model.Maximize(sum(objective_terms))

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Extract the solution
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        schedule = {}
        for i in range(num_tas):
            schedule[i] = [j for j in range(num_shifts) if solver.Value(x[i, j]) == 1]
        return schedule
    else:
        print("\nNo feasible solution found. Possible reasons:")
        print("- Check if total TA shift requirements match total shift needs.")
        print("- Ensure there are enough available TAs for each shift.")
        print("- Check if constraints are overly strict.")
        return None

# Example Input
availability = [
    [1, 0, -1, 1, 1],   # TA 0: Desired, Undesired, Unavailable, Desired, Desired
    [1, 1, 0, -1, 1],   # TA 1: Desired, Desired, Undesired, Unavailable, Desired
    [-1, 1, 1, 0, 1],   # TA 2: Unavailable, Desired, Desired, Undesired, Desired
]
ta_requirements = [2, 3, 2]  # TA 0: 2 shifts, TA 1: 1 shift, TA 2: 2 shifts
shift_requirements = [2, 1, 1, 1, 2]  # Shifts: 2 TAs, 1 TA, 1 TA, 1 TA, 2 TAs

# Solve
solution = schedule_tas_debug(availability, ta_requirements, shift_requirements)
if solution:
    print("\nFeasible Solution:")
    for ta, shifts in solution.items():
        print(f"TA {ta} assigned to shifts: {shifts}")
else:
    print("No feasible solution found.")


Availability Matrix:
TA 0: [1, 0, -1, 1, 1]
TA 1: [1, 1, 0, -1, 1]
TA 2: [-1, 1, 1, 0, 1]

TA Requirements: [2, 3, 2]
Shift Requirements: [2, 1, 1, 1, 2]
Added TA 0 shift requirement: 2
Added TA 1 shift requirement: 3
Added TA 2 shift requirement: 2
Added shift 0 staffing requirement: 2
Added shift 1 staffing requirement: 1
Added shift 2 staffing requirement: 1
Added shift 3 staffing requirement: 1
Added shift 4 staffing requirement: 2
TA 0 cannot work shift 2
TA 1 cannot work shift 3
TA 2 cannot work shift 0

Feasible Solution:
TA 0 assigned to shifts: [0, 3]
TA 1 assigned to shifts: [0, 1, 4]
TA 2 assigned to shifts: [2, 4]


## Class-based Implementation: Scheduler

### Data classes

In [29]:
@dataclass
class TA:
    id: int
    name: str
    req_shift_per_week: int
    availability: List[int] = field(default_factory=list)
    # to be planned
    assigned_shifts: List[int] = field(default_factory=list)

@dataclass
class Shift:
    id: int
    name: str
    series: str
    day_of_week: str
    start_time: time
    end_time: time
    req_ta_per_shift: int
    # to be planned
    assigned_tas: List[TA] = field(default_factory=list)


# @dataclass
# class TA:
#     _id_counter: int = field(init=False, repr=False, default=0)  # Static counter for IDs (class-level)
#     id: int = field(init=False)                                  # Auto-incremented ID
#     name: str
#     req_shift_per_week: int
#     availability: List[int] = field(default_factory=list)
#     assigned_shifts: List[int] = field(default_factory=list)

#     def __post_init__(self):
#         # Increment the counter and assign the ID
#         self.__class__._id_counter += 1
#         self.id = self.__class__._id_counter

# @dataclass
# class Shift:
#     _id_counter: int = field(init=False, repr=False, default=0)  # Static counter for IDs (class-level)
#     id: int = field(init=False)                                  # Auto-incremented ID
#     name: str
#     series: str
#     day_of_week: str
#     start_time: time
#     end_time: time
#     req_ta_per_shift: int
#     assigned_tas: List[TA] = field(default_factory=list)

#     def __post_init__(self):
#         # Increment the counter and assign the ID
#         self.__class__._id_counter += 1
#         self.id = self.__class__._id_counter


@dataclass
class Schedule:
    tas: List[TA] = field(default_factory=list)
    shifts: List[Shift] = field(default_factory=list)

### Solver Logic

In [24]:
class TA_Scheduling:
    def __init__(self, schedule):
        self.schedule = schedule
        # Solver configuration
        self.model = cp_model.CpModel()
        self.solver = cp_model.CpSolver()
        self.solver.parameters.linearization_level = 0
        self.solver.parameters.num_search_workers = os.cpu_count()
        # Properties to be computed
        self.assigment_matrix = {}

    def add_ta(self, ta: TA):
        self.schedule.tas.append(ta)

    def add_shift(self, shift: Shift):
        self.schedule.shifts.append(shift)

    def update_availability(self, ta_id: str, availability: List[int]):
        ta = next((ta for ta in self.schedule.tas if ta.id == ta_id), None)
        if ta:
            ta.availability = availability
        else:
            print(f"TA with ID {ta_id} not found.")

    def update_shift_requirements(self, shift_id: str, req_ta_per_shift: int):
        shift = next((shift for shift in self.schedule.shifts if shift.id == shift_id), None)
        if shift:
            shift.req_ta_per_shift = req_ta_per_shift
        else:
            print(f"Shift with ID {shift_id} not found.")

    def update_ta_requirements(self, ta_id: str, req_shift_per_week: int):
        ta = next((ta for ta in self.schedule.tas if ta.id == ta_id), None)
        if ta:
            ta.req_shift_per_week = req_shift_per_week
        else:
            print(f"TA with ID {ta_id} not found.")

    # 1 - Create Desicion Variables
    def create_decision_variables(self):
        for ta in self.schedule.tas:
            for shift in self.schedule.shifts:
                self.assigment_matrix[(ta.id, shift.id)] = self.model.NewBoolVar(f'availability_{ta.id}_{shift.id}')

    # 2 - Constraint Definitions
    # (2.1) TAs must meet their shift requirements
    def tas_meet_shift_requirements(self):
        for ta in self.schedule.tas:
            self.model.Add(sum(self.assigment_matrix[(ta.id, shift.id)] for shift in self.schedule.shifts) == ta.req_shift_per_week)

    # (2.2) Shifts must meet staffing requirements
    def shifts_meet_staffing_requirements(self):
        for shift in self.schedule.shifts:
            self.model.Add(sum(self.assigment_matrix[(ta.id, shift.id)] for ta in self.schedule.tas) == shift.req_ta_per_shift)

    # (2.3) Respect TA availability
    def respect_ta_availability(self):
      for ta in self.schedule.tas:
        for shift in self.schedule.shifts:
          if ta.availability[shift.id] == -1:  # Unavailable
            self.model.Add(self.assigment_matrix[(ta.id, shift.id)] == 0)

    # 3 - Define Objective function
    def objective_function(self):
        objective_terms = []
        for ta in self.schedule.tas:
            for shift in self.schedule.shifts:
                if ta.availability[shift.id] == 1:
                    objective_terms.append(10 * self.assigment_matrix[(ta.id, shift.id)])
                elif ta.availability[shift.id] == 0:
                    objective_terms.append(-1 * self.assigment_matrix[(ta.id, shift.id)])
        self.model.Maximize(sum(objective_terms))

    # Put everything together
    def solve(self):
        self.create_decision_variables()
        # Apply constraints
        self.tas_meet_shift_requirements()
        self.shifts_meet_staffing_requirements()
        self.respect_ta_availability()
        # Optional Objective function
        self.objective_function()

        self.solver.Solve(self.model)

        if self.solver.Status() == cp_model.OPTIMAL or self.solver.Status() == cp_model.FEASIBLE:
            print("\nFeasible Solution:")
            for ta in self.schedule.tas:
                ta_shifts = [shift for shift in self.schedule.shifts if self.solver.Value(self.assigment_matrix[(ta, shift)]) == 1]
                print(f"TA {ta.id} assigned to shifts: {ta_shifts}")
                ta.assigned_shifts = ta_shifts
        else:
            print("\nNo feasible solution found. Possible reasons:")
            print("- Check if total TA shift requirements match total shift needs.")
            print("- Ensure there are enough available TAs for each shift.")
            print("- Check if constraints are overly strict.")

        return self.solver.Status()

    def report(self):
      pass

### Testing

In [25]:
# Testing the class definitions above

shift1 = Shift(id= 1, name='Morning Shift',    series='A', day_of_week='Monday',     start_time=time(8, 0),  end_time=time(16, 0), req_ta_per_shift=2)
shift2 = Shift(id= 2, name='Afternoon Shift',  series='B', day_of_week='Tuesday',    start_time=time(14, 0), end_time=time(22, 0), req_ta_per_shift=1)
shift3 = Shift(id= 3, name='Evening Shift',    series='C', day_of_week='Wednesday',  start_time=time(18, 0), end_time=time(0, 0), req_ta_per_shift=2)
shift4 = Shift(id= 4, name='Night Shift',      series='D', day_of_week='Thursday',   start_time=time(0, 0),  end_time=time(8, 0),  req_ta_per_shift=1)
shift5 = Shift(id= 5, name='Late Night Shift', series='E', day_of_week='Friday',     start_time=time(20, 0), end_time=time(4, 0),  req_ta_per_shift=2)

ta1 = TA(id= 1, name='John Doe',       req_shift_per_week=1)
ta2 = TA(id= 2, name='Jane Smith',     req_shift_per_week=2)
ta3 = TA(id= 3, name='Alice Johnson',  req_shift_per_week=1)
ta4 = TA(id= 4, name='Bob Brown',      req_shift_per_week=2)

schedule = Schedule()
schedule.tas = [ta1, ta2, ta3, ta4]
schedule.shifts = [shift1, shift2, shift3, shift4, shift5]

ta_scheduling = TA_Scheduling(schedule)

In [27]:
ta_scheduling.update_availability(ta_id='1', availability=[1, 0, 1, 1,  1])
ta_scheduling.update_availability(ta_id='2', availability=[1, 1, 0, 1,  1])
ta_scheduling.update_availability(ta_id='3', availability=[1, 1, 1, 0,  1])
ta_scheduling.update_availability(ta_id='4', availability=[1, 1, 0, 0, -1])

In [28]:
ta_scheduling.solve()

TypeError: list indices must be integers or slices, not str

## Class-based Data Constructors
Constructs TA, and Shift objects from CSV files.

In [ ]:
# TODO
class Data_Constructor:
    def __init__(self, ta_csv_path, shift_csv_path, availability_folder):
        self.ta_csv_path = ta_csv_path
        self.shift_csv_path = shift_csv_path
        self.availability_folder = availability_folder
        # Data holders
        self.schedule = Schedule()
        self.ta_data    = None  # Placeholder for TA data
        self.shift_data = None  # Placeholder for Shift data
        # Construction routine
        self.load_data()
        self.validate_data()
        self.create_ta_objects()
        self.create_shift_objects()
        self.load_availabilities()


    def load_data(self):
        self.ta_data = pd.read_csv(self.ta_csv_path)
        self.shift_data = pd.read_csv(self.shift_csv_path)

    def create_ta_objects(self):
        for index, row in self.ta_data.iterrows():
            ta = TA(
                id=row['id'],
                name=row['name'],
                req_shift_per_week=row['req_shift_per_week']
            )
            self.schedule.tas.append(ta)

    def create_shift_objects(self):
        for index, row in self.shift_data.iterrows():
            shift = Shift(
                id=row['id'],
                name=row['name'],
                series=row['series'],
                day_of_week=row['day_of_week'],
                start_time=datetime.strptime(row['start_time'], '%H:%M').time(),
                end_time=datetime.strptime(row['end_time'], '%H:%M').time(),
                req_ta_per_shift=row['req_ta_per_shift']
            )
            self.schedule.shifts.append(shift)

    # Data validation
    def validate_data(self):
        return (self.validate_ta_availability() and self.validate_total_availability())

    def validate_ta_availability(self):
        for ta in self.schedule.tas:
            count_available = sum(1 for availability in ta.availability if availability != -1)
            if count_available < ta.req_shift_per_week:
                print(f"TA {ta.id} has insufficient availability.")
                print(f"Available shifts: {count_available}")
                print(f"Required shifts: {ta.req_shift_per_week}")
                return False
        return True

    def validate_total_availability(self):
        total = 0
        for ta in self.schedule.tas:
            count_available = sum(1 for availability in ta.availability if availability != -1)
            total += count_available
        return total >= sum(ta.req_shift_per_week for ta in self.schedule.tas)


# General Optimization Practice

## CP-SAT - Nurse Scheduling

In [ ]:
from ortools.sat.python import cp_model

### Data Creation

In [ ]:
num_nurses = 4
num_shifts = 3
num_days = 3
all_nurses = range(num_nurses)
all_shifts = range(num_shifts)
all_days = range(num_days)

### Create the model

In [ ]:
model = cp_model.CpModel()

### Create the variables
The array defines assignments for shifts to nurses as follows: shifts[(n, d, s)] equals 1 if shift s is assigned to nurse n on day d, and 0 otherwise.

In [ ]:
shifts = {}
for n in all_nurses:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d, s)] = model.new_bool_var(f"shift_n{n}_d{d}_s{s}")

### Code the constraints
Next, we show how to assign nurses to shifts subject to the following constraints:

Each shift is assigned to a single nurse per day.
Each nurse works at most one shift per day.

Here's the code that creates the first condition

In [ ]:
for d in all_days:
    for s in all_shifts:
        model.add_exactly_one(shifts[(n, d, s)] for n in all_nurses)

The last line says that for each shift, the sum of the nurses assigned to that shift is 1.

Next, here's the code that requires that each nurse works at most one shift per day.

In [ ]:
for n in all_nurses:
    for d in all_days:
        model.add_at_most_one(shifts[(n, d, s)] for s in all_shifts)

For each nurse, the sum of shifts assigned to that nurse is at most 1 ("at most" because a nurse might have the day off).

**Assign shifts evenly**

In [ ]:
# Try to distribute the shifts evenly, so that each nurse works
# min_shifts_per_nurse shifts. If this is not possible, because the total
# number of shifts is not divisible by the number of nurses, some nurses will
# be assigned one more shift.
min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
if num_shifts * num_days % num_nurses == 0:
    max_shifts_per_nurse = min_shifts_per_nurse
else:
    max_shifts_per_nurse = min_shifts_per_nurse + 1
for n in all_nurses:
    shifts_worked = []
    for d in all_days:
        for s in all_shifts:
            shifts_worked.append(shifts[(n, d, s)])
    model.add(min_shifts_per_nurse <= sum(shifts_worked))
    model.add(sum(shifts_worked) <= max_shifts_per_nurse)

Since there are num_shifts * num_days total shifts in the schedule period, you can assign at least (num_shifts * num_days) // num_nurses

shifts to each nurse, but some shifts may be left over. (Here // is the Python integer division operator, which returns the floor of the usual quotient.)

For the given values of num_nurses = 4, num_shifts = 3, and num_days = 3, the expression min_shifts_per_nurse has the value (3 * 3 // 4) = 2, so you can assign at least two shifts to each nurse. This is specified by the constraint (here in Python)

In [ ]:
model.add(min_shifts_per_nurse <= sum(shifts_worked))

Since there are nine total shifts over the three-day period, there is one remaining shift after assigning two shifts to each nurse. The extra shift can be assigned to any nurse.

The final line (here in Python)

In [ ]:
model.add(sum(shifts_worked) <= max_shifts_per_nurse)

### Update solver parameters

In [ ]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

### Register a callback function
You need to register a callback on the solver that will be called at each solution. (kind of optional)

In [ ]:
class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_nurses = num_nurses
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print(f"Solution {self._solution_count}")
        for d in range(self._num_days):
            print(f"Day {d}")
            for n in range(self._num_nurses):
                is_working = False
                for s in range(self._num_shifts):
                    if self.value(self._shifts[(n, d, s)]):
                        is_working = True
                        print(f"  Nurse {n} works shift {s}")
                if not is_working:
                    print(f"  Nurse {n} does not work")
        if self._solution_count >= self._solution_limit:
            print(f"Stop search after {self._solution_limit} solutions")
            self.stop_search()

    def solutionCount(self):
        return self._solution_count

# Display the first five solutions.
solution_limit = 5
solution_printer = NursesPartialSolutionPrinter(
    shifts, num_nurses, num_days, num_shifts, solution_limit
)

### Invoke the solver

In [ ]:
solver.solve(model, solution_printer)

### Entire nurse scheduling program

In [ ]:
"""Example of a simple nurse scheduling problem."""
from ortools.sat.python import cp_model


def main() -> None:
    # Data.
    num_nurses = 4
    num_shifts = 3
    num_days = 3
    all_nurses = range(num_nurses)
    all_shifts = range(num_shifts)
    all_days = range(num_days)

    # Creates the model.
    model = cp_model.CpModel()

    # Creates shift variables.
    # shifts[(n, d, s)]: nurse 'n' works shift 's' on day 'd'.
    shifts = {}
    for n in all_nurses:
        for d in all_days:
            for s in all_shifts:
                shifts[(n, d, s)] = model.new_bool_var(f"shift_n{n}_d{d}_s{s}")

    # Each shift is assigned to exactly one nurse in the schedule period.
    for d in all_days:
        for s in all_shifts:
            model.add_exactly_one(shifts[(n, d, s)] for n in all_nurses)

    # Each nurse works at most one shift per day.
    for n in all_nurses:
        for d in all_days:
            model.add_at_most_one(shifts[(n, d, s)] for s in all_shifts)

    # Try to distribute the shifts evenly, so that each nurse works
    # min_shifts_per_nurse shifts. If this is not possible, because the total
    # number of shifts is not divisible by the number of nurses, some nurses will
    # be assigned one more shift.
    min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
    if num_shifts * num_days % num_nurses == 0:
        max_shifts_per_nurse = min_shifts_per_nurse
    else:
        max_shifts_per_nurse = min_shifts_per_nurse + 1
    for n in all_nurses:
        shifts_worked = []
        for d in all_days:
            for s in all_shifts:
                shifts_worked.append(shifts[(n, d, s)])
        model.add(min_shifts_per_nurse <= sum(shifts_worked))
        model.add(sum(shifts_worked) <= max_shifts_per_nurse)

    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.parameters.linearization_level = 0
    # Enumerate all solutions.
    solver.parameters.enumerate_all_solutions = True

    class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
        """Print intermediate solutions."""

        def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
            cp_model.CpSolverSolutionCallback.__init__(self)
            self._shifts = shifts
            self._num_nurses = num_nurses
            self._num_days = num_days
            self._num_shifts = num_shifts
            self._solution_count = 0
            self._solution_limit = limit

        def on_solution_callback(self):
            self._solution_count += 1
            print(f"Solution {self._solution_count}")
            for d in range(self._num_days):
                print(f"Day {d}")
                for n in range(self._num_nurses):
                    is_working = False
                    for s in range(self._num_shifts):
                        if self.value(self._shifts[(n, d, s)]):
                            is_working = True
                            print(f"  Nurse {n} works shift {s}")
                    if not is_working:
                        print(f"  Nurse {n} does not work")
            if self._solution_count >= self._solution_limit:
                print(f"Stop search after {self._solution_limit} solutions")
                self.stop_search()

        def solutionCount(self):
            return self._solution_count

    # Display the first five solutions.
    solution_limit = 5
    solution_printer = NursesPartialSolutionPrinter(
        shifts, num_nurses, num_days, num_shifts, solution_limit
    )

    solver.solve(model, solution_printer)

    # Statistics.
    print("\nStatistics")
    print(f"  - conflicts      : {solver.num_conflicts}")
    print(f"  - branches       : {solver.num_branches}")
    print(f"  - wall time      : {solver.wall_time} s")
    print(f"  - solutions found: {solution_printer.solutionCount()}")


if __name__ == "__main__":
    main()

Solution 1
Day 0
  Nurse 0 does not work
  Nurse 1 works shift 0
  Nurse 2 works shift 1
  Nurse 3 works shift 2
Day 1
  Nurse 0 works shift 2
  Nurse 1 does not work
  Nurse 2 works shift 1
  Nurse 3 works shift 0
Day 2
  Nurse 0 works shift 2
  Nurse 1 works shift 1
  Nurse 2 works shift 0
  Nurse 3 does not work
Solution 2
Day 0
  Nurse 0 works shift 0
  Nurse 1 does not work
  Nurse 2 works shift 1
  Nurse 3 works shift 2
Day 1
  Nurse 0 does not work
  Nurse 1 works shift 2
  Nurse 2 works shift 1
  Nurse 3 works shift 0
Day 2
  Nurse 0 works shift 2
  Nurse 1 works shift 1
  Nurse 2 works shift 0
  Nurse 3 does not work
Solution 3
Day 0
  Nurse 0 works shift 0
  Nurse 1 does not work
  Nurse 2 works shift 1
  Nurse 3 works shift 2
Day 1
  Nurse 0 works shift 1
  Nurse 1 works shift 2
  Nurse 2 does not work
  Nurse 3 works shift 0
Day 2
  Nurse 0 works shift 2
  Nurse 1 works shift 1
  Nurse 2 works shift 0
  Nurse 3 does not work
Solution 4
Day 0
  Nurse 0 works shift 0
  Nurse 